In [1]:
from keras.utils import to_categorical
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pyplot as plt
import cv2
import struct
import os
import time
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
from keras.models import Sequential
from keras.layers import Conv2D, Dropout, MaxPooling2D, Flatten, Dense, Reshape, Lambda
from keras.models import load_model
from sklearn.decomposition import PCA
from tensorflow.keras.layers import Input, Conv3D, MaxPooling3D, UpSampling3D, UpSampling2D
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv3D, MaxPooling3D, UpSampling3D, LSTM, Dense, Conv2DTranspose
from tensorflow.keras.models import Model
from sklearn.metrics import mean_squared_error, accuracy_score, confusion_matrix
import pickle


In [2]:
from google.colab import drive
gpath = drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!unrar x "/content/drive/My Drive/Dataset/Depth.rar"


UNRAR 6.11 beta 1 freeware      Copyright (c) 1993-2022 Alexander Roshal


Extracting from /content/drive/My Drive/Dataset/Depth.rar

Creating    Depth                                                     OK
Extracting  Depth/a03_s06_e02_sdepth.bin                                   0%  OK 
Extracting  Depth/a05_s10_e01_sdepth.bin                                   0%  OK 
Extracting  Depth/a20_s10_e02_sdepth.bin                                   0%  OK 
Extracting  Depth/a10_s07_e02_sdepth.bin                                   0%  OK 
Extracting  Depth/a12_s04_e03_sdepth.bin                                   0%  OK 
Extracting  Depth/a17_s03_e01_sdepth.bin                                   0%  OK 
Extracting  Depth/a10_s10_e01_sdepth.bin                                   0%  OK 
Extracting  Depth/a09_s09_e03_sdepth.bin                                   1%  OK 
Extracting  Depth/a04_s10_e03_sdepth.bin                

In [4]:
print('Number of samples : ',len(os.listdir(path="/content/Depth/")))

dataset_path = "/content/Depth/"

Number of samples :  567


In [3]:
class_names = {
    "a01": "High Arm Wave",
    "a02": "Horizontal Arm Wave",
    "a03": "Hammer",
    "a04": "Hand Catch",
    "a05": "Forward Punch",
    "a06": "High Throw",
    "a07": "Draw X",
    "a08": "Draw Tick",
    "a09": "Draw Circle",
    "a10": "Hand Clap",
    "a11": "Two Hand Wave",
    "a12": "Side-Boxing",
    "a13": "Bend",
    "a14": "Forward Kick",
    "a15": "Side Kick",
    "a16": "Jogging",
    "a17": "Tennis Swing",
    "a18": "Tennis Serve",
    "a19": "Golf Swing",
    "a20": "Pick Up and Throw"
}


class_labels = {
    "a01": 1,
    "a02": 2,
    "a03": 3,
    "a04": 4,
    "a05": 5,
    "a06": 6,
    "a07": 7,
    "a08": 8,
    "a09": 9,
    "a10": 10,
    "a11": 11,
    "a12": 12,
    "a13": 13,
    "a14": 14,
    "a15": 15,
    "a16": 16,
    "a17": 17,
    "a18": 18,
    "a19": 19,
    "a20": 20
}

In [6]:
#initializing empty lists

samples_list = []

lables_list = []

files_names_list = os.listdir(path=dataset_path)

#change these numbers for batch size
start = 500
end = 567

i = 0
for filename in files_names_list:
    i = i + 1
    print("Reading File - " + str(i))
    print()

    if not filename.endswith("depth.bin"):
        continue # skip file that is not binary file

    # Appending class labels in label list

    class_name = filename.split("_")[0]
    lables_list.append(class_labels[class_name])


    # Appending corresponding video/image sequence samples in sample list

    # Loading Depth videos from binary files
    with open(dataset_path + filename, "rb") as f:

        frame_count,  = struct.unpack('i', f.read(4))  # frame count

        """print(f"Frame count is {frame_count}")
        print('------------------')
        print()"""

        width,  = struct.unpack('i', f.read(4))  # width
        height,  = struct.unpack('i', f.read(4))  # height

        image_sequence = []

        # Creating frame image from binary bytes
        for single_frame in range(frame_count):

          # Reading Image Buffer

          bytes_buffer = f.read(width*height*4)

          flatten_array = np.frombuffer(bytes_buffer, dtype=np.uint32)

          single_depth_img = flatten_array.reshape((240, 320))

          #complete video sequence from image
          image_sequence.append(single_depth_img)


    samples_list.append(image_sequence)


print()
print('Number of samples : ' + str(len(samples_list)))

print()

#print(len(lables_list))



Reading File - 1

Reading File - 2

Reading File - 3

Reading File - 4

Reading File - 5

Reading File - 6

Reading File - 7

Reading File - 8

Reading File - 9

Reading File - 10

Reading File - 11

Reading File - 12

Reading File - 13

Reading File - 14

Reading File - 15

Reading File - 16

Reading File - 17

Reading File - 18

Reading File - 19

Reading File - 20

Reading File - 21

Reading File - 22

Reading File - 23

Reading File - 24

Reading File - 25

Reading File - 26

Reading File - 27

Reading File - 28

Reading File - 29

Reading File - 30

Reading File - 31

Reading File - 32

Reading File - 33

Reading File - 34

Reading File - 35

Reading File - 36

Reading File - 37

Reading File - 38

Reading File - 39

Reading File - 40

Reading File - 41

Reading File - 42

Reading File - 43

Reading File - 44

Reading File - 45

Reading File - 46

Reading File - 47

Reading File - 48

Reading File - 49

Reading File - 50

Reading File - 51

Reading File - 52

Reading File - 53

Re

In [4]:
encoder_features_list_1 = pickle.load(open("/content/drive/MyDrive/features/autoencoder_samples/autoencoder_features_1.pkl", "rb"))
encoder_features_list_2 = pickle.load(open("/content/drive/MyDrive/features/autoencoder_samples/autoencoder_features_2.pkl", "rb"))
encoder_features_list_3 = pickle.load(open("/content/drive/MyDrive/features/autoencoder_samples/autoencoder_features_3.pkl", "rb"))
encoder_features_list_4 = pickle.load(open("/content/drive/MyDrive/features/autoencoder_samples/autoencoder_features_4.pkl", "rb"))
encoder_features_list_5 = pickle.load(open("/content/drive/MyDrive/features/autoencoder_samples/autoencoder_features_5.pkl", "rb"))


labels_list_1 = pickle.load(open("/content/drive/MyDrive/features/autoencoder_samples/feature_labels_1.pkl", "rb"))
labels_list_2 = pickle.load(open("/content/drive/MyDrive/features/autoencoder_samples/feature_labels_2.pkl", "rb"))
labels_list_3 = pickle.load(open("/content/drive/MyDrive/features/autoencoder_samples/feature_labels_3.pkl", "rb"))
labels_list_4 = pickle.load(open("/content/drive/MyDrive/features/autoencoder_samples/feature_labels_4.pkl", "rb"))
labels_list_5 = pickle.load(open("/content/drive/MyDrive/features/autoencoder_samples/feature_labels_5.pkl", "rb"))

temporal_feature_list = pickle.load(open("/content/drive/MyDrive/features/temporal_features.pkl", "rb"))

In [5]:
encoder_features_list = []

for feature in encoder_features_list_1:

  encoder_features_list.append(feature)

  pass



for feature in encoder_features_list_2:

  encoder_features_list.append(feature)

  pass



for feature in encoder_features_list_3:

  encoder_features_list.append(feature)

  pass



for feature in encoder_features_list_4:

  encoder_features_list.append(feature)

  pass



for feature in encoder_features_list_5:

  encoder_features_list.append(feature)

  pass



In [6]:
print(len(encoder_features_list))

567


In [7]:
print(sum(encoder_features_list[0][0])/len(encoder_features_list[0][0]))

0.46656070559401996


In [6]:
labels_list = []

for feature in labels_list_1:

  labels_list.append(feature)

  pass



for feature in labels_list_2:

  labels_list.append(feature)

  pass



for feature in labels_list_3:

  labels_list.append(feature)

  pass



for feature in labels_list_4:

  labels_list.append(feature)

  pass



for feature in labels_list_5:

  labels_list.append(feature)

  pass



In [9]:
print(len(labels_list))

567


In [10]:
print(len(temporal_feature_list))

567


In [7]:

sequence_features_list_1 = pickle.load(open("/content/drive/MyDrive/features/vgg_features_1.pkl", "rb"))
sequence_features_list_2 = pickle.load(open("/content/drive/MyDrive/features/vgg_features_2.pkl", "rb"))
sequence_features_list_3 = pickle.load(open("/content/drive/MyDrive/features/vgg_features_3.pkl", "rb"))
sequence_features_list_4 = pickle.load(open("/content/drive/MyDrive/features/vgg_features_4.pkl", "rb"))
sequence_features_list_5 = pickle.load(open("/content/drive/MyDrive/features/vgg_features_5.pkl", "rb"))
sequence_features_list_6 = pickle.load(open("/content/drive/MyDrive/features/vgg_features_6.pkl", "rb"))

In [8]:
sequence_features_list = []

for feature in sequence_features_list_1:

  sequence_features_list.append(feature)

  pass



for feature in sequence_features_list_2:

  sequence_features_list.append(feature)

  pass



for feature in sequence_features_list_3:

  sequence_features_list.append(feature)

  pass



for feature in sequence_features_list_4:

  sequence_features_list.append(feature)

  pass



for feature in sequence_features_list_5:

  sequence_features_list.append(feature)

  pass



for feature in sequence_features_list_6:

  sequence_features_list.append(feature)

  pass

In [9]:
print(len(sequence_features_list))

567


In [10]:
print(len(temporal_feature_list[111]))
print(len(sequence_features_list[111]))

#print(len(samples_list[111]))
print()

print(len(encoder_features_list[111]))

34
34

34


In [31]:
weighted_features_list = []


for i in range(len(sequence_features_list)):

  FD_i_dot_FM_i_list = []
  Sum_FD_i = sum(temporal_feature_list[i]) # Adding all frames difference (temporal features) togather for ith video/frame sequence sample to get Sum_FD_i

  autoencoder_frames_average_sum = []
  for j in range(len(sequence_features_list[i])):

    # multiplying each element (1,2,3,4 ...... k(1000)) of Complete spatial feature map FM_i (having length 1000) with weight temporal feature FD_i
    # to get N (45 frames for video 1) weighted arrays of length k(1000)

    #                         1000 elements (FM_i_(1-1000))       1 element (FD_i)
    #                               |    |                             |    |
    #                               |    |                             |    |
    #                               |    |                             |    |
    #                              \      /                           \      /
    #                               \    /                             \    /
    #                                \  /                               \  /
    #                                 \/                                 \/

    #FD_i_dot_FM_i_list.append(sequence_features_list[i][j] * temporal_feature_list[i][j] * (sum(encoder_features_list[i][j])/len(encoder_features_list[i][j])) )
    FD_i_dot_FM_i_list.append(sequence_features_list[i][j] * (sum(encoder_features_list[i][j])/len(encoder_features_list[i][j])) )

    autoencoder_frames_average_sum.append(sum(encoder_features_list[i][j])/len(encoder_features_list[i][j]))

    pass

  # Now applying summation (column wise) to all weighted arrays 'FD_i_dot_FM_i' in FD_i_dot_FM_i_list

  complete_FD_i_dot_FM_i_array = np.array(FD_i_dot_FM_i_list).sum(axis=0)

  # dividing every weighted sum element FM_i_dot_FD_i in complete array with sum_FD_i

  complete_FD_i_dot_FM_i_array = complete_FD_i_dot_FM_i_array/Sum_FD_i

  #complete_FD_i_dot_FM_i_array = complete_FD_i_dot_FM_i_array/sum(autoencoder_frames_average_sum)

  weighted_features_list.append(complete_FD_i_dot_FM_i_array)
  #break
  pass

print(len(weighted_features_list))


567


In [ ]:
for feature in weighted_features_list:

  print(len(feature))
  pass

In [32]:
print(len(labels_list))

567


In [33]:
X = weighted_features_list
Y = labels_list

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=42, train_size = .80)

In [34]:
print(len(X_train),len(X_test))

453 114


In [35]:
from sklearn.neural_network import MLPClassifier

classifier = MLPClassifier(hidden_layer_sizes=(2048,1024,512), max_iter=200, early_stopping=False,activation = 'relu',solver='adam',random_state=42, verbose = True)

classifier.fit(X_train, Y_train)

Iteration 1, loss = 3.67399642
Iteration 2, loss = 3.69743466
Iteration 3, loss = 3.07843682
Iteration 4, loss = 2.56230820
Iteration 5, loss = 2.20414992
Iteration 6, loss = 1.97888635
Iteration 7, loss = 1.59983007
Iteration 8, loss = 1.40619049
Iteration 9, loss = 1.09676210
Iteration 10, loss = 0.96242829
Iteration 11, loss = 0.78949714
Iteration 12, loss = 0.74122621
Iteration 13, loss = 0.59166644
Iteration 14, loss = 0.59094459
Iteration 15, loss = 0.47665100
Iteration 16, loss = 0.45107255
Iteration 17, loss = 0.35262389
Iteration 18, loss = 0.36573931
Iteration 19, loss = 0.36985948
Iteration 20, loss = 0.38684726
Iteration 21, loss = 0.32880053
Iteration 22, loss = 0.29035200
Iteration 23, loss = 0.16091036
Iteration 24, loss = 0.20707489
Iteration 25, loss = 0.13933171
Iteration 26, loss = 0.14336875
Iteration 27, loss = 0.09978133
Iteration 28, loss = 0.08758778
Iteration 29, loss = 0.09034790
Iteration 30, loss = 0.06424167
Iteration 31, loss = 0.04928595
Iteration 32, los

MLPClassifier(hidden_layer_sizes=(2048, 1024, 512), random_state=42,
              verbose=True)

In [36]:
predictions = classifier.predict(X_test)

def accuracy(confusion_matrix):
   diagonal_sum = confusion_matrix.trace()
   sum_of_all_elements = confusion_matrix.sum()
   return diagonal_sum / sum_of_all_elements

ConfusionMatrix = confusion_matrix(predictions, Y_test)

print("Accuracy of MLPClassifier : \n")

print(accuracy(ConfusionMatrix))

Accuracy of MLPClassifier : 

0.9298245614035088
